In [ ]:
database_path = r"C:\Users\User\Documents\University\Research\FakeNews\CoronaVirusProject\data\twitter_posts_30-12-2019_22-03-2020.db"
output_path = r'C:\Users\User\Documents\University\Research\FakeNews\CoronaVirusProject\data\twitter_location_analysis\twitter_posts_30-12-2019_22-03-2020'

In [ ]:
import os

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
import sqlite3 as sql
import time

T = time.time()

conn = sql.connect(database_path)
cur = conn.cursor()
query = "SELECT post_id, author FROM posts WHERE date > date('2019-12-31')"
tweet_authors = cur.execute(query).fetchall()

T = time.time() - T
print('Tweet IDs retrieved in {} seconds'.format(T))

In [ ]:
print('Got {} tweets with their authors'.format(len(tweet_authors)))

In [ ]:
authors = [tweet[1] for tweet in tweet_authors]

In [ ]:
author_parsed_locations = {}
for author in set(authors):
    query = "SELECT location FROM authors WHERE name='{}'".format(author)
    location = cur.execute(query).fetchall()
    author_parsed_locations[author] = location
    
fixed_author_parsed_locations = {}
for author, location in author_parsed_locations.items():
    if location:
        if location[0][0]:
            fixed_author_parsed_locations[author] = location[0][0]

In [ ]:
fixed_tweet_authors = [(tweet_id, author) for (tweet_id, author) in tweet_authors if author in fixed_author_parsed_locations]

In [ ]:
tweet_locations = [(tweet_id, fixed_author_parsed_locations[author]) for (tweet_id, author) in fixed_tweet_authors]

In [ ]:
import os
import json

country_name_dict = {}

if os.path.isfile('{}/country_name_dict.json'.format(output_path)):
    with open('{}/country_name_dict.json'.format(output_path), 'r') as file_handle:
        country_name_dict = json.load(file_handle)

In [ ]:
state_name_dict = {}

if os.path.isfile('{}/state_name_dict.json'.format(output_path)):
    with open('{}/state_name_dict.json'.format(output_path), 'r') as file_handle:
        state_name_dict = json.load(file_handle)

In [ ]:
tweet_countries = {}
if os.path.isfile('{}/tweet_countries.json'.format(output_path)):
    with open('{}/tweet_countries.json'.format(output_path), 'r') as file_handle:
        tweet_countries = json.load(file_handle)

tweet_states = {}
if os.path.isfile('{}/tweet_states.json'.format(output_path)):
    with open('{}/tweet_states.json'.format(output_path), 'r') as file_handle:
        tweet_states = json.load(file_handle)

In [ ]:
bad_country_names = set()
if os.path.isfile('{}/bad_country_names.json'.format(output_path)):
    with open('{}/bad_country_names.json'.format(output_path), 'r') as file_handle:
        bad_country_names = set(json.load(file_handle))

bad_state_names = set()
if os.path.isfile('{}/bad_state_names.json'.format(output_path)):
    with open('{}/bad_state_names.json'.format(output_path), 'r') as file_handle:
        bad_state_names = set(json.load(file_handle))

In [ ]:
state_exceptions = {}
if os.path.isfile('{}/state_exceptions.json'.format(output_path)):
    with open('{}/state_exceptions.json'.format(output_path), 'r') as file_handle:
        state_exceptions = json.load(file_handle)

In [ ]:
def add_state_exception(parsed_location, state):
    state_exceptions[parsed_location] = state
    with open('{}/state_exceptions.json'.format(output_path), 'w') as file_handle:
        json.dump(state_exceptions, file_handle)

In [ ]:
geolocator = Bing(api_key="AgSzclXa0ydRjeopKMp_qtSEob_A6_LHG8-bq2RMnGIadjGRPRUKANZiKlKdLw4I")

In [ ]:
from geopy.geocoders import Bing
from geopy.exc import GeocoderTimedOut, GeocoderQueryError, GeocoderQuotaExceeded, GeocoderServiceError
import pickle
import time

def get_country_name_by_bing_api(parsed_location, country_type):
    if country_type != 'country' and country_type != 'state':
        print('Country type must be either country state')
        return "", True
    if not parsed_location.strip():
        return "", True
    if parsed_location in bad_country_names:
        return "", True    
    if country_type == 'state' and parsed_location in bad_state_names:
        return "", True
    if country_type == 'state':
        for state_exception in state_exceptions:
            if state_exception in parsed_location.lower():
                return state_exceptions[state_exception], True
    if country_type == 'country' and parsed_location in country_name_dict:
        return country_name_dict[parsed_location], True
    if country_type == 'state' and parsed_location in state_name_dict:
        return state_name_dict[parsed_location], True
    
    try:
        location = geolocator.geocode(parsed_location)
        if location is None: # fail
            bad_country_names.add(parsed_location)
            return "", False
        if 'address' not in location.raw:
            bad_country_names.add(parsed_location)
            return "", False
        if 'countryRegion' not in location.raw['address']:
            bad_country_names.add(parsed_location)
            return "", False
        country_name = location.raw['address']['countryRegion']
        country_name_dict[parsed_location] = country_name
        if country_name == 'United States':
            if 'adminDistrict' not in location.raw['address']:
                bad_state_names.add(parsed_location)
                if country_type == 'state':
                    return "", False
            else:
                state = location.raw['address']['adminDistrict']
                state_name_dict[parsed_location] = state
                if country_type == 'state':
                    return state, False
        else:
            if country_type == 'state':
                return "", False
        if country_type == 'country':
            return country_name, False
        else:
            bad_country_names.add(parsed_location)
            return "", False

    except GeocoderTimedOut as e:
        print(e)
        time.sleep(2)
        try:
            location = geolocator.geocode(parsed_location)
        except GeocoderTimedOut as e2:
            success = False
            while not success:
                time.sleep(5)
                try:
                    location = geolocator.geocode(parsed_location)
                    success = True
                except GeocoderTimedOut as e3:
                    pass
        if location is None: # fail
            bad_country_names.add(parsed_location)
            return "", False
        if 'address' not in location.raw:
            bad_country_names.add(parsed_location)
            return "", False
        if 'countryRegion' not in location.raw['address']:
            bad_country_names.add(parsed_location)
            return "", False
        country_name = location.raw['address']['countryRegion']
        country_name_dict[parsed_location] = country_name
        if country_name == 'United States':
            if 'adminDistrict' not in location.raw['address']:
                bad_state_names.add(parsed_location)
                print('ATTENTION: {} added to bad states set'.format(parsed_location))
                if country_type == 'state':
                    return "", False
            else:
                state = location.raw['address']['adminDistrict']
                state_name_dict[parsed_location] = state
                if country_type == 'state':
                    return state, False
        else:
            if country_type == 'state':
                return "", False
        if country_type == 'country':
            return country_name, False
        else:
            bad_country_names.add(parsed_location)
            return "", False
    except GeocoderQueryError as e:
        bad_country_names.add(parsed_location)
        print('Query error! query: {}'.format(parsed_location))
        return "Bad_Request", False

    except GeocoderQuotaExceeded as e:
        print('Quota exceeded!')
        time.sleep(1.5)
        return "Bad_Request", False
    
    except GeocoderServiceError as e:
        print('Service error!')
        time.sleep(1)
        return "Bad_Request", False
        

In [ ]:
bad_requests = 0
num_requests_total = 0
start_time = time.time()
for i, (tweet, location) in enumerate(tweet_locations, 1):
    if tweet in tweet_countries:
        continue
    country_name, is_cached = get_country_name_by_bing_api(location, 'country')
    if not is_cached:
        num_requests_total += 1
        if num_requests_total % 500 == 0:
            with open('{}/country_name_dict.json'.format(output_path), 'w') as handle:
                json.dump(country_name_dict, handle)
            with open('{}/state_name_dict.json'.format(output_path), 'w') as handle:
                json.dump(state_name_dict, handle)
            with open('{}/bad_country_names.json'.format(output_path), 'w') as handle:
                json.dump(list(bad_country_names), handle)
            with open('{}/bad_state_names.json'.format(output_path), 'w') as handle:
                json.dump(list(bad_state_names), handle)

            print("Dictionary was saved! ")

    if country_name == 'Bad_Request':
        bad_requests += 1
    else:
        if country_name:
            tweet_countries[tweet] = country_name
    if i % 10000 == 0:
        print('Finished {} tweets. So far {} requests total. {} seconds total.'.format(i, num_requests_total, time.time() - start_time))

print('Finished {} tweets. {} bad requests.'.format(len(tweet_locations), bad_requests))

with open('{}/country_name_dict.json'.format(output_path), 'w') as handle:
    json.dump(country_name_dict, handle)
with open('{}/state_name_dict.json'.format(output_path), 'w') as handle:
    json.dump(state_name_dict, handle)
with open('{}/bad_country_names.json'.format(output_path), 'w') as handle:
    json.dump(list(bad_country_names), handle)
with open('{}/bad_state_names.json'.format(output_path), 'w') as handle:
    json.dump(list(bad_state_names), handle)
    
with open('{}/tweet_countries.json'.format(output_path), 'w') as handle:
    json.dump(tweet_countries, handle)
    
print("Dictionary was saved!")


In [ ]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent='-')
def geolocate(country):
    try:
        loc = geolocator.geocode(country)
        return (loc.latitude, loc.longitude)
    except:
        return np.nan

In [ ]:
import json

with open('{}/tweet_countries.json'.format(output_path), 'w') as file_handle:
    json.dump(tweet_countries, file_handle)

In [ ]:
country_list = [country for (tweet_id, country) in tweet_countries.items()]
country_names = sorted(list(set([country for (_, country) in tweet_countries.items()])))
name_coordinate_map = {}
for country in country_names:
    latlon = geolocate(country)
    if latlon != latlon: # if latlon is nan
        continue
    name_coordinate_map[country] = latlon

In [ ]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

def get_continent(col):
    try:
        cn_a2_code = country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown'
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown'
    return (cn_a2_code, cn_continent)

In [ ]:
import json

def get_alpha3_code(alpha2_code):
    with open('{}/country_alpha_codes.json'.format(output_path), 'r') as file_handle:
        country_codes_list = json.load(file_handle)    
    for country in country_codes_list:
        if country['alpha2'].lower() == alpha2_code.lower():
            return country['alpha3'].upper()
    return 'INVALID'

In [ ]:
import pandas as pd

country_tweet_count_tup_list = [(country, country_list.count(country)) for country in name_coordinate_map.keys()]
df = pd.DataFrame({'country_name' : [country for (country, c_count) in country_tweet_count_tup_list],
                  'tweet_count' : [c_count for (country, c_count) in country_tweet_count_tup_list],
                  'alpha-2-code' : [get_continent(country)[0] for (country, _) in country_tweet_count_tup_list],
                   'alpha-3-code' : [get_alpha3_code(get_continent(country)[0]) for (country, _) in country_tweet_count_tup_list],
                  'continent_code': [get_continent(country)[1] for (country, _) in country_tweet_count_tup_list]})

In [ ]:
df['latitude'] = [name_coordinate_map[country][0] for (country, _) in country_tweet_count_tup_list]
df['longitude'] = [name_coordinate_map[country][1] for (country, _) in country_tweet_count_tup_list]

In [ ]:
df.head()

In [ ]:
import folium
from folium.plugins import MarkerCluster

world_map = folium.Map(titles="cartodbpositron")
folium.Choropleth('{}/world_countries.json'.format(output_path), fill_color='Reds',
                    data=df, columns=['alpha-3-code', 'tweet_count'], key_on='feature.id', nan_fill_color='white',
                     legend_name='Tweet count', fill_opacity=0.7, line_opacity=0.2).add_to(world_map)
marker_cluster = MarkerCluster().add_to(world_map)

for i in range(len(df)):
    lat = df.iloc[i]['latitude']
    long = df.iloc[i]['longitude']
    popup_text = 'Country: {}<br> #Tweets: {}'.format(df.iloc[i]['country_name'], df.iloc[i]['tweet_count'])
    folium.CircleMarker(location = [lat, long], radius=5, popup=popup_text, fill=True).add_to(marker_cluster)
    
world_map.save('{}/world_map.html'.format(output_path))
world_map

In [ ]:
df = df.drop(df[df['country_name'] == 'United States'].index)

world_map = folium.Map(titles="cartodbpositron")
folium.Choropleth('{}/world_countries.json'.format(output_path), fill_color='Reds',
                    data=df, columns=['alpha-3-code', 'tweet_count'], key_on='feature.id', nan_fill_color='white',
                     legend_name='Tweet count', fill_opacity=0.7, line_opacity=0.2).add_to(world_map)
marker_cluster = MarkerCluster().add_to(world_map)

for i in range(len(df)):
    lat = df.iloc[i]['latitude']
    long = df.iloc[i]['longitude']
    popup_text = 'Country: {}<br> #Tweets: {}'.format(df.iloc[i]['country_name'], df.iloc[i]['tweet_count'])
    folium.CircleMarker(location = [lat, long], radius=5, popup=popup_text, fill=True).add_to(marker_cluster)
    
world_map.save('{}/world_map_no_us.html'.format(output_path))
world_map

In [ ]:
def get_topn_countries(df, topn):
    country_tweet_count_list = [(df.iloc[i]['country_name'], count) for (i, count) in enumerate(list(df['tweet_count']))]
    sorted_list = sorted(country_tweet_count_list, key=lambda tup: tup[1], reverse=True)
    return sorted_list[:topn]

get_topn_countries(df, 10)

In [ ]:
country_count_dict_with_us = {country_name : country_list.count(country_name) for country_name in set(country_list)}

In [ ]:
top10_country_count_dict_with_us = sorted(list(country_count_dict_with_us.items()), reverse=True, key=lambda tup: tup[1])[:10]
fig, ax = plt.subplots(figsize=(25, 10))
ax.set_ylabel('Number of tweets')
ax.set_title('Tweets per country')
plt.bar([country for (country, _) in top10_country_count_dict_with_us], [count for (_, count) in top10_country_count_dict_with_us])
plt.show()

## States

In [ ]:
tweet_locations_dict = {tweet_id : location for (tweet_id, location) in tweet_locations}
us_tweet_locations = {tweet_id : tweet_locations_dict[tweet_id] for (tweet_id, tweet_country) in tweet_countries.items() if tweet_country == 'United States'}

In [ ]:
bad_requests = 0
num_requests_total = 0
start_time = time.time()
for i, (tweet, location) in enumerate(us_tweet_locations.items(), 1):
    if tweet in tweet_states:
        continue

    state_name, is_cached = get_country_name_by_bing_api(location, 'state')
    if not is_cached:
        num_requests_total += 1
    if state_name == 'Bad_Request':
        bad_requests += 1
    else:
        if state_name:
            tweet_states[tweet] = state_name
    if i % 10000 == 0:
        print('Finished {} tweets. So far {} requests total. {} seconds total.'.format(i, num_requests_total, time.time() - start_time))
print('Finished {} tweets. {} bad requests.'.format(len(tweet_states), bad_requests))

In [ ]:
# fix arbitrary bing errors
for tweet in tweet_states:
    if tweet_states[tweet] == 'Nevada':
        tweet_states[tweet] = 'NV'
    if tweet_states[tweet] == 'New York':
        tweet_states[tweet] = 'NY'

In [ ]:
with open('{}/tweet_states.json'.format(output_path), 'w') as file_handle:
    json.dump(tweet_states, file_handle)

In [ ]:
state_list = [state for (_, state) in tweet_states.items()]

In [ ]:
state_counts = {}
for state in set(state_list):
    state_counts[state] = state_list.count(state)

In [ ]:
state_counts_list = list(state_counts.items())
states_df = pd.DataFrame({'state_code': [state for (state, _) in state_counts_list], 'tweet_count': [count for (_, count) in state_counts_list]})

In [ ]:
geolocator = Nominatim(user_agent='-')

with open('{}/state_code_name.json'.format(output_path)) as file_handle:
    state_code_names = json.load(file_handle)

In [ ]:
state_coordinates = []

for state_code in [state_code for (state_code, _) in state_counts_list]:
    state_name = state_code_names[state_code]
    if state_name == 'Washington':
        state_name = 'Washington state'
    try:
        location = geolocator.geocode(state_name)
        state_coordinates.append((location.latitude, location.longitude))
    except:
        print('COULD NOT GEOCODE: {}'.format(state))

In [ ]:
states_df['latitude'] = [coordinates[0] for coordinates in state_coordinates]
states_df['longitude'] = [coordinates[1] for coordinates in state_coordinates]

In [ ]:
import folium
from folium.plugins import MarkerCluster

state_map = folium.Map(titles="cartodbpositron")
folium.Choropleth('{}/us_states.json'.format(output_path), fill_color='Reds',
                    data=states_df, columns=['state_code', 'tweet_count'], key_on='feature.id', nan_fill_color='white',
                     legend_name='Tweet count', fill_opacity=0.7, line_opacity=0.2).add_to(state_map)
marker_cluster = MarkerCluster().add_to(state_map)

for i in range(len(states_df)):
    lat = states_df.iloc[i]['latitude']
    long = states_df.iloc[i]['longitude']
    popup_text = 'State: {}<br> #Tweets: {}'.format(state_code_names[states_df.iloc[i]['state_code']], states_df.iloc[i]['tweet_count'])
    folium.CircleMarker(location = [lat, long], radius=5, popup=popup_text, fill=True).add_to(marker_cluster)
    
state_map.save('{}/state_map.html'.format(output_path))
state_map

In [ ]:
def get_topn_states(df, topn):
    state_tweet_count_list = [(state_code_names[df.iloc[i]['state_code']], count) for (i, count) in enumerate(list(df['tweet_count']))]
    sorted_list = sorted(state_tweet_count_list, key=lambda tup: tup[1], reverse=True)
    return sorted_list[:topn]

get_topn_states(states_df, 10)

In [ ]:
import matplotlib.pyplot as plt

state_counts = get_topn_states(states_df, 15)

fig, ax = plt.subplots(figsize=(25, 10))
ax.set_ylabel('Number of tweets')
ax.set_title('Tweets per state')
plt.bar([state for (state, _) in state_counts], [count for (_, count) in state_counts])
plt.show()

In [ ]:
## temporary
import json

with open(r'D:\iliapl\topic_modeling\data\output_data\POI_Followers_13-06-20_PERSON_ONLY_V10_TOP40PERCENT_25TOPICS\tweet_author_map.json', 'r') as f:
    tweet_author_map = json.load(f)

In [ ]:
with open(r'D:\iliapl\topic_modeling\data\output_data\twitter_location_analysis\POI_Followers_13-06-20\tweet_countries.json', 'r') as f:
        tweet_countries = json.load(f)

In [ ]:
sum = 0
for tweet in tweet_countries:
    if tweet in tweet_author_map and tweet_countries[tweet] == 'New Zealand':
        sum += 1

In [ ]:
sum

In [ ]:
author_countries = {}
for tweet in tweet_author_map:
    author = tweet_author_map[tweet]
    if author not in author_countries and tweet in tweet_countries:
        author_countries[author] = tweet_countries[tweet]

In [ ]:
author_country_count = {}
for author in author_countries:
    country = author_countries[author]
    author_country_count[country] = author_country_count.get(country, 0) + 1

In [ ]:
import matplotlib.pyplot as plt

country_count_tuples = sorted(list(author_country_count.items()), reverse=True, key=lambda tup: tup[1])[20:30]

plt.figure(figsize=(15, 10))
plt.bar([tup[0] for tup in country_count_tuples], [tup[1] for tup in country_count_tuples], align='center')
plt.title('Number of Authors per Country')
#plt.xticks(range(len(author_country_count)), country_count_tuples)

plt.show()

In [ ]:
author_country_count['New Zealand']